## Ensemble: Bagging x Boosting x Stacking

### Conteúdo dessa aula:
- Tipos de Ensemble
- Bagging: Random Forest

Ensemble nada mais é do que a **sabedoria da maioria**. Aqui combinamos vários modelos mais simples em um único 
modelo robusto a fim de reduzir o viés, variância e/ou aumentar a acurácia.
<br>


## Tipos de Ensemble:
- __1. Bagging (short for bootstrap aggregation)__: Treina paralelamente N modelos mais fracos (geralmente do mesmo tipo - homogêneo) com __N subsets distintos__ criados com __amostragem randômica e reposição (bootstrap)__. Cada modelo é avaliado na fase de teste com o label definido pela moda (classificação) ou pela __média dos valores__ (regressão). Devido à essa agregação final que vem o aggregation do nome. Os métodos de Bagging reduzem a variância da predição. <br>
Algoritmos  famosos: Random Forest <br>
<img src='images/bagging.png' style="width:600px"  text="http://cheatsheets.aqeel-anwar.com" />  
<br>
<br>

- __2. Boosting__: Treina N modelos mais fracos (geralmente do mesmo tipo - homogênio) de forma sequencial. Os pontos que foram classificados erroneamente recebem um peso maior para entrar no próximo modelo. Na fase de teste, cada modelo é avaliado com base do erro de teste de cada modelo, a predição é feita com um peso sobre a votação. Os métodos de Boosting reduzem o viés da predição. <br>
Algoritmos  famosos: AdaBoost, Gradient Boosting, XGBoost, CatBoost, LightGBM (Light Gradient Boosting Machine) <br>
<img src='images/boosting.png' style="width:600px" text="Fonte: http://cheatsheets.aqeel-anwar.com" />
<br>
<br>

- __3. Ensemble de modelos distintos__: Treina N modelos distintos, por exemplo: um Random Forest e um SVM e faz a previsão de acordo com a saída desses modelos. 
<br>
<br>


- __4. Stacking__: Treina N modelos mais fracos (geralmente de tipos distintos - heterogênio) em um subset do conjunto de dados. Uma vez que os modelos foram treinados, cria-se um novo modelo (meta learning) para combinar a saída de cada um dos modelos mais fracos resultando na predição final. Isso é feito no segundo subset dos dados. Na fase de teste, cada modelo mais fraco faz sua predição independentemente e esses labels entram como features do meta learner para gerar a predição final.
<br>
<img src='images/stacking.png' style="width:600px" text="Fonte: http://cheatsheets.aqeel-anwar.com" />
<br>
<br>

##### Resumo:
<img src='images/comparison_img.png' style="width:600px" text="Fonte: https://quantdare.com/what-is-the-difference-between-bagging-and-boosting" />

<img src='images/comparison.png' style="width:600px" />


## Bagging: [Random Forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

Uma técnica muito interessante baseada em árvores é o **Random Forest**. O random forest faz um ensemble de modelos de árvore de decisão, mas com duas melhorias: 

> Selecionamos **aleatoriamente com reposição** algumas linhas da base original. Isso gera um novo dataset (reamostrado), chamado de **bootstrapped dataset**. O número de linhas do dataset reamostrado é controlável.

> __Cada vez que fazemos um split das árvores de decisão, apenas uma amostra aleatória das features é comparada para escolher o split__. A quantidade de features a serem consideradas é controlável. (max_features e bootstrap_features). 

<img src="https://www.researchgate.net/profile/Nikolaos-Sapountzoglou/publication/339447755/figure/fig1/AS:862073311469568@1582545702096/Example-of-a-random-forest.png" width=800>

Dessa forma, cada árvore será treinada em um dataset diferente (devido ao bootstrap) e assim, cada modelo cometerá erros em diferentes lugares gerando um viés e uma variância diferente para cada um. 

Esse processo é denominado de **bootstrapping** e ele introduz **duas fontes de aleatoriedade**, cujo objetivo é **diminuir a variância** (tendência a overfitting) do modelo.

De fato, árvores individuais são facilmente overfitadas, como discutimos em aula (lembre-se da grande flexibilidade da hipótese em encontrar condições favoráveis à aprendizagem dos ruídos!).

Com esta aleatorização introduzida pelo bootstrapping, o objetivo é que as árvores construídas sejam **independentes**, de modo que **os erros cometidos por cada uma sejam independentes** e dessa forma as RFs atacam o principal problema das DTs: a variância.

Deste modo, se considerarmos as previsões isoladas e de alguma forma **agregar** as previsões, a expectativa é que o modelo final seja **menos propenso a overfitting**! Mas, uma pergunta natural é: o que é essa "agregação"? Aqui entra o segundo elemento do bagging...

Como cada árvore produz **o seu target**, a **agregação** é utilizada para tomar a decisão final:

> No caso de classificação, a classe final é atribuída como **a classe majoritária**, isso é, **a classe que foi o output $\hat{y}$ mais vezes dentre todas as árvores**;

> No caso de regressão, o valor final é atribuído como **a média dos valores preditos $\hat{y}$ por cada árvore**.

Note que em ambos os casos, o procedimento de agregação pode ser visto como uma **média**, e o sklearn deixa isso explícito: "*In contrast to the original publication, the scikit-learn implementation combines classifiers by averaging their probabilistic prediction, instead of letting each classifier vote for a single class.*"

Tomando a média como procedimento de agregação, a expectativa é que **alguns erros sejam anulados**, garantindo uma previsão final **mais estável e mais generalizável**, dado que os ruídos são eliminados.

Esses dois processos juntos, bootstrap e agregation, são nomeados de bagging.

No final de todo o processo de construção das árvores, as features mais importantes terão uma probabilidade maior de aparecer próximas às raízes das árvores (root), enquanto features menos importantes aparecerão próximas aos nós finais (leaves). Dessa forma, é possível estimar a importância de uma feature calulando a profundidade média em que ela aparece ao longo de todas as árvores.

Agora pensem: é ruim termos duas variáveis muito correlacionas nesse tipo de modelagem?

#### Pré-processamento dos dados

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("../data/bank-full.csv") 
df.head()

Attribute Information:

   - age (numeric)
   - job : type of job (categorical: admin.', 'bluecollar', 'entrepreneur', 'housemaid', 'management', 'retired', 'selfemployed', 'services', 'student', 'technician', 'unemployed', 'unknown')
   - marital : marital status (categorical:'divorced', 'married', 'single', 'unknown'; note: 'divorced' means divorced or widowed)
   - education (categorical: 'basic.4y', 'basic.6y', 'basic.9y', 'high.school', 'illiterate', 'professional.course', 'university.degree', 'unknown')
   - default: has credit in default? (categorical: 'no','yes','unknown')
   - balance: average yearly balance, in euros (numeric)
   - housing: has housing loan? (categorical: 'no','yes','unknown')
   - loan: has personal loan? (categorical: 'no','yes','unknown')
   - contact: contact communication type (categorical:'cellular','telephone','unknown')
   - day: last contact day of the month (numeric 1 -31)
   - month: last contact month of year (categorical: 'jan', 'feb','mar', …, 'nov', 'dec')
   - duration: last contact duration, in seconds (numeric).
    Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known.Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
   - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
   - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
   - previous: number of contacts performed before this campaign and for this client (numeric)
   - poutcome: outcome of the previous marketing campaign (categorical: 'failure', 'nonexistent', 'success')
   - target: has the client subscribed a term deposit? (binary:"yes", "no")


In [ ]:
#### Separar feature e target
y = df['Target'].copy()
x = df.drop('Target', axis=1).copy()

#### particionar dados
from sklearn.model_selection import train_test_split 
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=0.2, random_state=42) 

In [ ]:
# Resetar os index
x_train, x_test, y_train, y_test = x_train.reset_index(drop=True), x_test.reset_index(drop=True), y_train.reset_index(drop=True), y_test.reset_index(drop=True)

In [ ]:
# Selecionar as colunas categóricas e numéricas
cat_columns = x_train.select_dtypes(['object']).columns
num_columns = x_train.select_dtypes(exclude=['object']).columns
cat_columns

In [ ]:
# Printa valores de cada coluna categórica
[print(f"{c}: {x_train[c].unique()}") for c in cat_columns]

In [ ]:
# Converte colunas categóricas em numéricas
# Importa OneHotEncoder e LabelEncoder


# Intância ambos


# Enconding das features categóricas


# Converte para pandas dataframe renomeando as colunas


# Concatena colunas do encoder com as numéricas


# Encoding dos labels


In [ ]:
# Temos classes desbalanceadas?


[Random Forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

In [ ]:
# Vamos importar o modelo de classificação do Random Forest


In [ ]:
# Instancia a classe do modelo
model_rf = 

# Faz o treino na base de treino


# Faz a predição do modelo treinado na base de teste
y_pred_rf = 

In [ ]:
# Obter parâmetros utilizados no treino do modelo


In [ ]:
# Obter probabilidades estimadas para cada classe


#### Avaliar o modelo

In [ ]:
# Importa classes que serão utilizadas


# Gera o classification_report para o teste


In [ ]:
# Calcula a confusion_matrix para o teste


# Plota a confusion matrix


## Feature Importance

In [ ]:
features = x_train.columns
importances = model_rf.feature_importances_
indices = np.argsort(importances)

plt.figure(figsize=(10,10))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

O que aconteceria aqui se tivéssemos classes muito correlacionadas?

Podemos criar modelos menos complexos selecionando as features mais importantes:

In [ ]:
importance = ['duration', 'balance', 'age', 'day', 'poutcome_success', 'pdays', 'campaign', 'housing_yes', 'previous']

# Instancia modelo
model_rfi = 

# Faz o fit considerando apenas as variáveis mais importantes


# Faz o predict apenas nas variáveis mais importantes
y_pred_rfi =

# Classification report



Com um modelo bem menos complexo conseguimos um resultado próximo do modelo no qual utilizamos todas as features.

_____________________________________________
_____________________________________________
_____________________________________________

### Random Search

___________
___________
___________

### Como ficaria esse código com Pipeline

In [ ]:
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as pp
from imblearn.under_sampling import RandomUnderSampler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import KFold, RandomizedSearchCV, GridSearchCV, train_test_split, cross_validate
from imblearn.under_sampling import RandomUnderSampler

x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=0.2, random_state=42) 

importance_pipe = ['duration', 'balance', 'age', 'day', 'poutcome', 'pdays', 'campaign', 'housing', 'previous']
cat_columns_pipe = [c for c in cat_columns if c in importance_pipe]

# Encoding dos labels
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

# Criando pipeline das variáveis categóricas
cat_pipe = Pipeline([
    ('ohe', OneHotEncoder(drop='first', sparse=False, handle_unknown='ignore'))
])

# Para termos no modelo tanto as variáveis categóricas quanto as numéricas
preprocessor = ColumnTransformer([
    ('cat', cat_pipe, cat_columns_pipe)
], remainder='passthrough')

# Criando pipeline final
pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestClassifier(random_state=42))
])

# Tunando hiperparâmetros com 3-fold cross-validation e pipelines
parameters = {'model__max_depth': [3, 4, 5],
              "model__n_estimators" : [5, 8, 10],
              "model__criterion" : ["gini", "entropy"],}

kfold = KFold(n_splits=3, shuffle=True, random_state=42)
# grid = GridSearchCV(pipe, param_grid=parameters, cv=kfold, scoring='f1', n_jobs=2)
grid = RandomizedSearchCV(pipe, param_distributions=parameters, cv=kfold, scoring='f1', n_jobs=2)
grid.fit(x_train[importance_pipe], y_train)

# qual o melhor parâmetro
grid.best_params_ 


In [ ]:
# Predizendo no x_test
y_pred = grid.predict(x_test[importance_pipe])

print(classification_report(y_test, y_pred))

Podemos predizer em uma única amostra de dado

In [ ]:
my_data = x_test[importance_pipe].iloc[[77]]
y_pred_ = grid.predict(my_data)
print(my_data)
print(y_pred_)

Se quisermos fazer um tratamento de classes desbalanceadas, precisamos utilizar o Pipeline do pacote imblearn:

In [ ]:
from imblearn.pipeline import Pipeline as pp
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE

steps = [('over', SMOTE()), 
         ('scale', MinMaxScaler()),
         ('model', RandomForestClassifier())]
pipeline = pp(steps=steps)
scores = cross_validate(pipeline, x_train[num_columns], y_train, scoring='roc_auc', cv=3, n_jobs=-1)
scores

In [ ]:
# Tunando hiperparâmetros com 3-fold cross-validation e pipelines
parameters = {'model__max_depth': [3, 4, 5],
              "model__n_estimators" : [100, 150, 200],
              "model__criterion" : ["gini", "entropy"],}

grid_imbalanced = RandomizedSearchCV(pipeline, param_distributions=parameters, cv=3, scoring='f1', n_jobs=2)

# Vamos usar só variáveis numéricas por causa do SMOTE
importance_pipe_num = [c for c in importance if c in num_columns]

grid_imbalanced.fit(x_train[importance_pipe_num], y_train)

# qual o melhor parâmetro
grid_imbalanced.best_params_ 

____________________
______________________
________________

## Vantagens e Desvantagens
__Vantagens:__
* Geralmente fornecem modelos com alta acurácia (classificação)
* Necessita de pouco tratamento dos dados
* Conseguem lidar com dados faltantes (dois métodos: média dos valores para repor variáveis contínuas e computa proximity-weighted average)
* Fornece uma estimativa da importancia das features
* São robustos aos [outliers](https://stats.stackexchange.com/questions/187200/how-are-random-forests-not-sensitive-to-outliers) nas variáveis independentes e conseguem lidar com eles automaticamente (tendem a isolar os outliers)
* Podem ser usados na seleção de features
* Conseguem construir fronteiras de decisão não-lineares
* Os dados não precisam seguir uma distribuição normal (como em modelos lineares), mas vale testar!
* Lidam bem com uma quantidade muito grande de dados e features

__Desvantagens:__
* No RF as variáveis categóricas não ordinais devem ser convertidas para dummies ([mean encoding](https://towardsdatascience.com/why-you-should-try-mean-encoding-17057262cd0))
* Não é recomendado utilizar nenhum modelo que depende do bagging em problemas com classes desbalanceados. Em dados extremamente desbalanceados existe uma probabilidade significativa de uma amostra ser selecionada com poucas ou nenhuma amostra da classe minoritária.
* Não é muito recomendado na extrapolação de dados (suponha a predição de preço de casas. Se sua regressão linear foi treinada com casas até 4 quartos e no teste aparece uma de 8 você consegue extrapolar e prever de forma consistente. Com os modelos de árvore não.)
* Em geral, não lidam bem com dados muito esparsos.
* É menos interpretável que uma regressão linear ou uma árvore de decisão, por exemplo.


## Bibliografia e Aprofundamento
- [Problemas com as variáveis dummies](https://towardsdatascience.com/one-hot-encoding-is-making-your-tree-based-ensembles-worse-heres-why-d64b282b5769)
- [When to avoid RF](https://stats.stackexchange.com/questions/112148/when-to-avoid-random-forest)
- [Very skewed data](https://stats.stackexchange.com/questions/172842/best-practices-with-data-wrangling-before-running-random-forest-predictions)
- [Como RF lida com missing](https://www.numpyninja.com/post/all-about-random-forests-and-handling-missing-values-in-them)
- [Feature Importance x Feature Permutation](https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html#sphx-glr-auto-examples-ensemble-plot-forest-importances-py)
- [Out-of-bag error](https://en.wikipedia.org/wiki/Out-of-bag_error)
- [OOB error vs. test set error](https://uc-r.github.io/random_forests)
- [Principais dúvidas no XGBoost](https://towardsdatascience.com/20-burning-xgboost-faqs-answered-to-use-the-library-like-a-pro-f8013b8df3e4)

## Exercício
Utilize o dataset "data/german_credit_data.csv" para implementar o Random Forest e o XGBoost.


In [ ]:
df = pd.read_csv("../data/german_credit_data.csv", index_col=0)

X = df.drop(columns="Risk")
y = df["Risk"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
